In [1]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.verifier import TextualVerifierWithTracker

In [2]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
initial_solution = """
To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

question = Variable("Solve 3x^2-7x+2=0",
                    requires_grad=False,
                    role_description="math question")

instruction = Variable("""You will answering a math question. 
                        Please using step-by-step explanation. Be super concise.""",
                        requires_grad=False,
                        role_description="instruction")

# Calculation Using Steps
calculation = Variable(initial_solution,
                        requires_grad=True,
                        role_description="solution to the math question")


In [4]:

verification_task_prompts = [
    """
    Verify whether the calculation step is correct. 
    If there is a mistake in the step, provide ONLY the corrected version of the calculation step. 
    Do NOT solve the entire equation unless an error is found.
    """,
    """
    As a senior mathematician, carefully examine the calculation step for correctness. 
    If any error is found, revise and present only the corrected version of that specific step. 
    Avoid solving the entire problem unless a correction is necessary
    """
]

verifier = TextualVerifierWithTracker(
    verifier_engine=engine, 
    use_cot_generation=True,
    use_step_breakdown=True,
    verification_task_prompts=verification_task_prompts,
    enable_logging=True
)

verified_result = verifier.verify(instance=question,
                                  instruction=instruction,
                                  calculation=calculation)

print("VERIFIED RESULT:", verified_result.value)

INFO:textgrad:TextualVerifier: Start verification process...
INFO:textgrad:TextualVerifier: Ready to verify 8 calculation steps...
INFO:textgrad:TextualVerifier: Verifying step 1/8...
INFO:textgrad:TextualVerifier: Generating step 1 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 1 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 1...
INFO:textgrad:TextualVerifier: Verifying step 2/8...
INFO:textgrad:TextualVerifier: Generating step 2 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 2 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 2...
INFO:textgrad:TextualVerifier: Verifying step 3/8...
INFO:textgrad:TextualVerifier: Generating step 3 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 3 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 3...
INFO:textgrad:TextualVerifier: Verifying step 4/8...
INFO:textgrad:TextualVerifier: Generating step 4 variant 1/2...
IN

In [5]:
import json

tracker_data = verifier.get_tracker()

with open('tracker_results/raw_verification_with_cot_and_steps.json', 'w') as f:
    json.dump(tracker_data, f, indent=4, default=str)

In [6]:
tracker_data

{'setup': {'verifier_engine': <textgrad.engine.gemini.ChatGemini at 0x122eb9d90>,
  'use_cot_generation': True,
  'use_step_breakdown': True,
  'verification_task_prompts': ['\n    Verify whether the calculation step is correct. \n    If there is a mistake in the step, provide ONLY the corrected version of the calculation step. \n    Do NOT solve the entire equation unless an error is found.\n    ',
   '\n    As a senior mathematician, carefully examine the calculation step for correctness. \n    If any error is found, revise and present only the corrected version of that specific step. \n    Avoid solving the entire problem unless a correction is necessary\n    '],
  'enable_logging': True},
 'variable': {'instance': Variable(value=Solve 3x^2-7x+2=0, role=math question, grads=set()),
  'instruction': Variable(value=You will answering a math question. 
                          Please using step-by-step explanation. Be super concise., role=instruction, grads=set()),
  'calculation': Va